In [45]:
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import dateutil

def parse(line):
    elems = line.split(' ')
    
    obj = {
        'timestamp': dateutil.parser.parse(elems[0])
    }
    for elem in elems[1:]: 
        (key,v) = elem.split(':')
        obj[key] = v
        
    return obj

def extract_byte(x, index):
    def byte_mapper(x):
        if not isinstance(x, basestring):
            return None
        if len(x) > 0:
            return ord(x.decode('hex')[index])
        else:
            return None
    return x.map(byte_mapper)

lines = open('all.txt', "rt").readlines()
column_order = ['timestamp', 'ID1', 'PTYPE', 'SEQ', 'ID2', 'CON', 'B9', 'BLEN', 'MTYPE', 'BODY', 'CRC']
packets = pd.DataFrame(map(lambda x: parse(x), lines), columns=column_order).set_index('timestamp')
packets.head()

,ID1,PTYPE,SEQ,ID2,CON,B9,BLEN,MTYPE,BODY,CRC
timestamp,,,,,,,,,,
2016-06-17 20:23:30.966899,1f014829,POD,25,1f014829,NaN,28,10,1d18,02a82800002b7bff8134,45\n
2016-06-17 20:24:44.634776,1f014829,POD,28,1f014829,NaN,30,10,1d18,02a82800002b7fff0360,ea\n
2016-06-17 20:25:53.328967,1f014829,POD,31,1f014829,NaN,38,10,1d18,02a8a800002b87ff037c,f2\n
2016-06-17 20:48:44.551830,1f014829,POD,02,1f014829,NaN,00,10,1d18,02ad2800002bdfff009c,df\n
2016-06-17 20:48:47.828112,1f014829,POD,05,1f014829,NaN,08,10,1d18,02ad2800002bdfff83c5,cf\n


In [46]:
# Drop packets that have incorrect body length; likely corrupt packets
packets['blen_actual'] = packets['BODY'].map(lambda x: len(x) / 2, 'ignore').fillna(0).astype(np.int)
packets['BLEN'] = packets['BLEN'].map(lambda x: int(x), 'ignore').fillna(0)
ok_big_packets = (packets['blen_actual'] == 23) & (packets['BLEN'] > 23)
no_body_packets = (packets['BLEN'] == 0)
ok_small_packets = (packets['blen_actual'] == packets['BLEN']) & (packets['blen_actual'] > 0)
packets = packets.loc[ok_big_packets | no_body_packets | ok_small_packets]
packets = packets.drop('blen_actual', axis=1)

# Drop sequential duplicates
packets = packets[packets["SEQ"].shift(-1) != packets["SEQ"]]


In [47]:
packets.loc[packets["ID1"] == 'ffffffff']

,ID1,PTYPE,SEQ,ID2,CON,B9,BLEN,MTYPE,BODY,CRC
timestamp,,,,,,,,,,
2016-06-30 20:52:58.930388,ffffffff,PDM,14,ffffffff,NaN,00,6.0,0704,1f0148288315,ec\n
2016-06-30 20:52:58.974498,ffffffff,POD,15,ffffffff,NaN,04,23.0,0115,02070002070002020000a5180005304e8d1f01482800c7,8c\n
2016-06-30 20:52:58.998169,ffffffff,ACK,16,1f014828,NaN,NaN,0.0,NaN,NaN,30\n
2016-06-30 20:52:59.787923,ffffffff,PDM,17,ffffffff,NaN,00,21.0,0313,1f0148281404061e1014340000a5180005304e82ea,03\n
2016-06-30 20:52:59.865898,ffffffff,POD,18,ffffffff,NaN,04,29.0,011b,13881008340a5002070002070002030000a5180005304e,59\n
2016-06-30 20:52:59.870056,ffffffff,ACK,19,1f014828,NaN,NaN,0.0,NaN,NaN,96\n
2016-06-30 20:52:59.997560,ffffffff,CON,20,NaN,1f0148280281,NaN,0.0,NaN,NaN,e7\n
2016-06-30 20:53:00.002081,ffffffff,ACK,21,1f014828,NaN,NaN,0.0,NaN,NaN,dd\n


### Look for body crc
(body crc is different from the 'CRC' column (packet crc) in the dataframe)

We're focused on the last two bytes of the body, as they seem to change like a crc.

* byte -2: bit 7 and bits 1:0 toggle, but 6:2 are always 0
* byte -1: two quiet bits: 6 and 0

In [48]:
# Let's compare packets of the same length, first
packets['BLEN'].value_counts()

10.0     370
0.0      370
3.0      279
50.0       6
31.0       5
25.0       3
24.0       3
12.0       3
205.0      2
32.0       2
21.0       2
6.0        2
126.0      1
29.0       1
23.0       1
Name: BLEN, dtype: int64

### xor each packet against all others, to identify bits that change (bit_deltas)

In [64]:
packets.loc[packets['BLEN'] == 12]

,ID1,PTYPE,SEQ,ID2,CON,B9,BLEN,MTYPE,BODY,CRC
timestamp,,,,,,,,,,
2016-06-30 20:53:00.680811,1f014828,PDM,22,1f014828,NaN,08,12.0,190a,c9fbbd584c0001f401028238,a4\n
2016-06-30 20:53:01.439286,1f014828,PDM,25,1f014828,NaN,10,12.0,190a,dde7b9467837000508028167,f8\n
2016-06-30 20:53:57.973529,1f014828,PDM,1,1f014828,NaN,20,12.0,190a,08e7fbe2380010a403028188,b3\n


In [65]:
import itertools

# Compare packets
body_packets = packets.loc[packets['BLEN'] == 12]

# (re)assemble raw packet data

ptype_values = {
    'PDM': 0b101,
    'POD': 0b111,
    'ACK': 0b010,
    'CON': 0b100,
    }

blen_hex = body_packets["BLEN"].map(lambda x: chr(int(x)).encode('hex'))
ptype_val = body_packets["PTYPE"].map(lambda x: ptype_values[x] << 5)
seq_val = body_packets["SEQ"].map(lambda x: int(x))
b5_hex = (ptype_val + seq_val).map(lambda x: chr(x).encode('hex'))
crc_hex = body_packets["CRC"].map(lambda x: x.rstrip())
header_hex_data = body_packets["ID1"] + b5_hex
raw_hex_data = body_packets["ID2"] + body_packets["B9"] + blen_hex + body_packets["MTYPE"] + body_packets["BODY"]

print (header_hex_data + raw_hex_data + crc_hex).head()
raw_data = raw_hex_data.map(lambda x: x.decode('hex'))

def xor_strings(s1,s2):
    xored_bytes = [ord(a) ^ ord(b) for a,b in zip(s1,s2)]
    return [s1.encode('hex'),s2.encode('hex')] + list(np.unpackbits(np.array(xored_bytes, dtype=np.uint8)))

combinations = map(lambda x: xor_strings(x[0], x[1]), itertools.combinations(list(raw_data), 2))
bit_deltas = pd.DataFrame(combinations)

#
crc_split_point = -16
bit_cols = list(range(0,len(combinations[0])-2))
body_columns = map(lambda x: ('body', x), bit_cols[:crc_split_point])
crc_columns = map(lambda x: ('crc', x), bit_cols[crc_split_point:])
                
bit_deltas.columns = pd.MultiIndex.from_tuples([('data', 'p1'), ('data', 'p2')] + body_columns + crc_columns)
bit_deltas['num_body_deltas'] = bit_deltas['body'].sum(axis=1)
bit_deltas['num_crc_deltas'] = bit_deltas['crc'].sum(axis=1)

# Yes, this is unwieldy, but necessary to be able to scan columns visually
pd.options.display.max_columns = 200
bit_deltas.head()

timestamp
2016-06-30 20:53:00.680811    1f014828b61f014828080c190ac9fbbd584c0001f40102...
2016-06-30 20:53:01.439286    1f014828b91f014828100c190adde7b946783700050802...
2016-06-30 20:53:57.973529    1f014828a11f014828200c190a08e7fbe2380010a40302...
dtype: object


data  \
                                         p1   
0  1f014828080c190ac9fbbd584c0001f401028238   
1  1f014828080c190ac9fbbd584c0001f401028238   
2  1f014828100c190adde7b9467837000508028167   

                                            body                             \
                                         p2    0  1  2  3  4  5  6  7  8  9   
0  1f014828100c190adde7b9467837000508028167    0  0  0  0  0  0  0  0  0  0   
1  1f014828200c190a08e7fbe2380010a403028188    0  0  0  0  0  0  0  0  0  0   
2  1f014828200c190a08e7fbe2380010a403028188    0  0  0  0  0  0  0  0  0  0   

                                                                              \
  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34   
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1   
2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1   

                                                                              \
  35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59   
0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   

                                                                              \
  60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84   
0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  1  1  1  0  0  0  0  0  0  0   
1  0  0  0  0  1  1  0  0  0  0  0  1  0  0  0  1  1  1  0  0  0  1  0  0  0   
2  0  0  0  0  1  1  0  1  0  1  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0   

                                                                            \
  85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106   
0  1  0  0  0  0  0  1  1  1  1  0  0  0  1  1   0   1   0   0   0   0   1   
1  1  1  0  1  0  1  1  1  0  1  0  0  1  1  1   0   1   0   0   0   0   0   
2  0  1  0  1  0  1  0  0  1  0  0  0  1  0  0   0   0   0   0   0   0   1   

                                                                               \
  107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125   
0   1   0   1   1   1   0   0   0   0   0   0   0   1   1   1   1   1   0   0   
1   0   0   0   0   0   0   0   0   1   0   0   0   1   0   1   0   1   0   0   
2   1   0   1   1   1   0   0   0   1   0   0   0   0   1   0   1   0   0   0   

                                                                          crc  \
  126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144   
0   0   1   0   0   0   0   1   0   0   1   0   0   0   0   0   0   0   0   0   
1   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   
2   0   1   0   0   0   0   1   0   1   1   0   0   0   0   0   0   0   0   0   

                                                              num_body_deltas  \
  145 146 147 148 149 150 151 152 153 154 155 156 157 158 159                   
0   0   0   0   0   0   1   1   0   1   0   1   1   1   1   1              28   
1   0   0   0   0   0   1   1   1   0   1   1   0   0   0   0              25   
2   0   0   0   0   0   0   0   1   1   1   0   1   1   1   1              25   

  num_crc_deltas  
                  
0              8  
1              5  
2              7

In [56]:
# This selects the number of 'collisions', or different frames with same crc.
# It could be used to identify bits not part of crc

(bit_deltas[(bit_deltas["num_body_deltas"] != 0) & (bit_deltas["num_crc_deltas"] == 0)])


Empty DataFrame
Columns: [(data, p1), (data, p2), (body, 0), (body, 1), (body, 2), (body, 3), (body, 4), (body, 5), (body, 6), (body, 7), (body, 8), (body, 9), (body, 10), (body, 11), (body, 12), (body, 13), (body, 14), (body, 15), (body, 16), (body, 17), (body, 18), (body, 19), (body, 20), (body, 21), (body, 22), (body, 23), (body, 24), (body, 25), (body, 26), (body, 27), (body, 28), (body, 29), (body, 30), (body, 31), (body, 32), (body, 33), (body, 34), (body, 35), (body, 36), (body, 37), (body, 38), (body, 39), (body, 40), (body, 41), (body, 42), (body, 43), (body, 44), (body, 45), (body, 46), (body, 47), (body, 48), (body, 49), (body, 50), (body, 51), (body, 52), (body, 53), (body, 54), (body, 55), (body, 56), (body, 57), (body, 58), (body, 59), (body, 60), (body, 61), (body, 62), (body, 63), (body, 64), (body, 65), (body, 66), (body, 67), (body, 68), (body, 69), (body, 70), (body, 71), (crc, 72), (crc, 73), (crc, 74), (crc, 75), (crc, 76), (crc, 77), (crc, 78), (crc, 79), (crc, 80), (crc, 81), (crc, 82), (crc, 83), (crc, 84), (crc, 85), (crc, 86), (crc, 87), (num_body_deltas, ), (num_crc_deltas, )]
Index: []

In [54]:
# Try reshuffling crc bits, since we have bits 6:2 zeroed out in second to last byte
# Are those bits padding? Or?
def pack_crc_bits(hex_str):
    data = hex_str.decode('hex')
    pb = ord(data[-2])
    pb = (pb & 0b11) #+ (pb >> 5)
    data = data[:-2] + chr(pb) + data[-1]
    return data.encode('hex')
map(pack_crc_bits, raw_hex_data.head().values)

['1f01482a10030e0100002c',
 '1f01482a18030e01000322',
 '1f01482a20030e01000096',
 '1f01482a28030e01000398',
 '1f01482a30030e010003be']

In [59]:

# Look at low change rows
ones = bit_deltas[(bit_deltas["num_body_deltas"] == 1)]
map(lambda x: (x, x % 16, ones.loc[ones[('body',x)] == 1]["crc"].iloc[0:1].values), range(0,ones["body"].shape[1]))
#ones.loc[ones[('body',34)] == 1]["data"]


[(0, 0, array([], shape=(0, 16), dtype=int64)),
 (1, 1, array([], shape=(0, 16), dtype=int64)),
 (2, 2, array([], shape=(0, 16), dtype=int64)),
 (3, 3, array([], shape=(0, 16), dtype=int64)),
 (4, 4, array([], shape=(0, 16), dtype=int64)),
 (5, 5, array([], shape=(0, 16), dtype=int64)),
 (6, 6, array([], shape=(0, 16), dtype=int64)),
 (7, 7, array([], shape=(0, 16), dtype=int64)),
 (8, 8, array([], shape=(0, 16), dtype=int64)),
 (9, 9, array([], shape=(0, 16), dtype=int64)),
 (10, 10, array([], shape=(0, 16), dtype=int64)),
 (11, 11, array([], shape=(0, 16), dtype=int64)),
 (12, 12, array([], shape=(0, 16), dtype=int64)),
 (13, 13, array([], shape=(0, 16), dtype=int64)),
 (14, 14, array([], shape=(0, 16), dtype=int64)),
 (15, 15, array([], shape=(0, 16), dtype=int64)),
 (16, 0, array([], shape=(0, 16), dtype=int64)),
 (17, 1, array([], shape=(0, 16), dtype=int64)),
 (18, 2, array([], shape=(0, 16), dtype=int64)),
 (19, 3, array([], shape=(0, 16), dtype=int64)),
 (20, 4, array([], shape